In [ ]:
# -----------------------------------------------------------------------------
# BatteryMind – Real-Time Prediction Demo
# -----------------------------------------------------------------------------
# Cell 1 – Imports & Endpoint Config
import json, time, boto3, numpy as np, pandas as pd
import plotly.express as px
from ai_models.inference.pipelines.real_time_inference import (
    RealTimeInferencePipeline
)
ENDPOINT_NAME = "BatteryMind-Transformer-Endpoint"
sagemaker = boto3.client("sagemaker-runtime")

# Cell 2 – Stream Simulator (reads synthetic telemetry & yields rows)
def stream_sensor_rows(csv_path, delay=0.5):
    for row in pd.read_csv(csv_path, chunksize=1):
        yield row.iloc[0].to_dict()
        time.sleep(delay)

# Cell 3 – Inference Loop
pipe = RealTimeInferencePipeline(endpoint_name=ENDPOINT_NAME)

predictions = []
for record in stream_sensor_rows("training-data/synthetic_datasets/battery_telemetry.csv"):
    payload = json.dumps(record)
    pred = pipe.predict(payload)
    predictions.append({**record, **pred})
    # Live plot every 100 rows
    if len(predictions) % 100 == 0:
        df_live = pd.DataFrame(predictions[-100:])
        fig = px.line(df_live, x="timestamp", y="soh_pred", title="Live SoH Prediction")
        fig.show()

# Cell 4 – Alerting Example
from ai_models.monitoring.alerts.alert_manager import AlertManager
am = AlertManager()
for p in predictions:
    if p["soh_pred"] < 0.7:
        am.create_alert(
            alert_type="LOW_SOH",
            severity="high",
            message=f"Battery {p['battery_id']} SoH below threshold: {p['soh_pred']:.2f}"
        )
